In [ ]:
# Face Cropping

import os
import cv2
lst = ["shivam"]
for i in lst:

  source_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}'
  destination_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}_Face'

  os.makedirs(destination_folder, exist_ok=True)

  def crop_faces(input_folder, output_folder, face_size=(128, 128)):
      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

      for img_name in os.listdir(input_folder):
          img_path = os.path.join(input_folder, img_name)

          img = cv2.imread(img_path)
          if img is None:
              continue

          gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

          faces = face_cascade.detectMultiScale(gray, 1.1, 4)

          for (x, y, w, h) in faces:
              face_img = img[y:y + h, x:x + w]
              resized_face = cv2.resize(face_img, face_size)
              output_path = os.path.join(output_folder, img_name)
              cv2.imwrite(output_path, resized_face)
              break

  crop_faces(source_folder, destination_folder)

  print("Face cropping completed. Cropped images saved in the destination folder.")


Face cropping completed. Cropped images saved in the destination folder.


In [ ]:
import os
import cv2

lst = ["Animesh", "Ankita", "Anmol", "Deependra", "Narayan", "Pankaj", "Sahil", "Sonal", "Suraj", "Vidya"]

for i in lst:
    source_folder = f'/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/{i}'
    destination_folder = f'/content/drive/Shareddrives/CNN_Project/Photos2026/{i}_Face'
    os.makedirs(destination_folder, exist_ok=True)

    def crop_faces_dnn(input_folder, output_folder, face_size=(128, 128), padding=10):
        # Provide the correct paths to the model and config files
        modelFile = '/content/drive/Shareddrives/CNN_Project/FaceModel/res10_300x300_ssd_iter_140000.caffemodel'
        configFile = '/content/drive/Shareddrives/CNN_Project/FaceModel/deploy.prototxt'
        net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

        for img_name in os.listdir(input_folder):
            img_path = os.path.join(input_folder, img_name)
            img = cv2.imread(img_path)

            if img is None:
                print(f"Could not read image {img_name}")
                continue

            h, w = img.shape[:2]
            blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], False, False)

            # Detect faces
            net.setInput(blob)
            detections = net.forward()

            for i in range(detections.shape[2]):
                confidence = detections[0, 0, i, 2]
                if confidence > 0.5:  # Adjust threshold for better results
                    box = detections[0, 0, i, 3:7] * [w, h, w, h]
                    (x1, y1, x2, y2) = box.astype("int")

                    # Add padding
                    x1 = max(0, x1 - padding)
                    y1 = max(0, y1 - padding)
                    x2 = min(w, x2 + padding)
                    y2 = min(h, y2 + padding)

                    # Crop face and resize
                    face_img = img[y1:y2, x1:x2]
                    resized_face = cv2.resize(face_img, face_size)

                    # Save cropped face
                    output_path = os.path.join(output_folder, img_name)
                    cv2.imwrite(output_path, resized_face)
                    print(f"Cropped face saved: {output_path}")
                    break  # Process only the first face detected

    crop_faces_dnn(source_folder, destination_folder)
    print(f"Face cropping completed for {i}. Cropped images saved in the destination folder.")


Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20240407_134438794.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240328_105548.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20240705_201413287.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/1676639882315.png
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20231223_112904.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/Polish_20231225_112744894.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240321_184308.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240303_153544.jpg
Cropped face saved: /content/drive/Shareddrives/CNN_Project/Photos2026/Animesh_Face/IMG_20240525_125222.jpg
Cropped face sav

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install facenet-pytorch


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from facenet_pytorch import InceptionResnetV1

# Base class to define the basic setup of the model
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self, x):
        raise NotImplementedError("Forward method not implemented!")




In [ ]:
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
class FaceRecognitionModel(BaseModel):
    def __init__(self, num_classes):
        super(FaceRecognitionModel, self).__init__()
        self.facenet = InceptionResnetV1(pretrained='vggface2').eval()
        for param in self.facenet.parameters():
            param.requires_grad = False

        # Convolutional layers
        self.conv1 = nn.Conv2d(512, 256, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)

        # Placeholder for fc1, updated dynamically
        self.fc1 = None
        self.fc2 = nn.Linear(256, num_classes)  # fc2 remains the same

    def initialize_fc1(self, input_shape):
        # Create a dummy input with the correct shape and device
        dummy_input = torch.randn(input_shape).to(next(self.parameters()).device)

        with torch.no_grad():
            dummy_output = self.facenet(dummy_input)

        # Calculate the flattened size after passing through the convolutional layers
        conv1_output = self.conv1(dummy_output.unsqueeze(-1).unsqueeze(-1))
        conv2_output = self.conv2(conv1_output)
        flat_size = conv2_output.view(conv2_output.size(0), -1).size(1)

        # Initialize fc1 with the calculated input size
        self.fc1 = nn.Linear(flat_size, 256).to(next(self.parameters()).device)




    def forward(self, x):
        if self.fc1 is None:
            raise ValueError("fc1 is not initialized. Call `initialize_fc1` with the input shape.")

        # Move input to the same device as the model parameters
        x = x.to(next(self.parameters()).device)

        with torch.no_grad():
            x = self.facenet(x)

        x = self.conv1(x.unsqueeze(-1).unsqueeze(-1))
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



    def fine_tune(self, train_loader, val_loader, num_epochs=10, lr=0.001):
        # Define loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        for epoch in range(num_epochs):
            self.train()
            train_loss = 0.0

            for images, labels in train_loader:
                # Move data to the same device as the model
                device = next(self.parameters()).device
                images, labels = images.to(device), labels.to(device)

                optimizer.zero_grad()
                outputs = self(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * images.size(0)

            # Validation loop
            self.eval()
            val_loss = 0.0
            correct = 0
            total = 0

            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = self(images)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs, 1)
                    correct += (predicted == labels).sum().item()
                    total += labels.size(0)

            # Print statistics
            train_loss /= len(train_loader.dataset)
            val_loss /= len(val_loader.dataset)
            val_accuracy = 100 * correct / total

            print(f"Epoch {epoch + 1}/{num_epochs}, "
                  f"Train Loss: {train_loss:.4f}, "
                  f"Validation Loss: {val_loss:.4f}, "
                  f"Validation Accuracy: {val_accuracy:.2f}%")




In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_to_idx = {}

        for idx, class_name in enumerate(os.listdir(root_dir)):
            self.class_to_idx[class_name] = idx
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                self.images.append(os.path.join(class_dir, img_name))
                self.labels.append(idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load training and validation data
train_dataset = FaceDataset('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train', transform=transform)
val_dataset = FaceDataset('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/val', transform=transform)



In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")

Model fc1 layer initialized successfully!


In [ ]:
# Fine-tune the model
model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")

Epoch 1/10, Train Loss: 2.3845, Validation Loss: 2.3532, Validation Accuracy: 18.18%
Epoch 2/10, Train Loss: 2.2367, Validation Loss: 2.0711, Validation Accuracy: 27.27%
Epoch 3/10, Train Loss: 1.6969, Validation Loss: 1.3986, Validation Accuracy: 67.27%
Epoch 4/10, Train Loss: 0.9851, Validation Loss: 0.6241, Validation Accuracy: 85.45%
Epoch 5/10, Train Loss: 0.4453, Validation Loss: 0.2616, Validation Accuracy: 96.36%
Epoch 6/10, Train Loss: 0.2370, Validation Loss: 0.1243, Validation Accuracy: 96.36%
Epoch 7/10, Train Loss: 0.1464, Validation Loss: 0.0893, Validation Accuracy: 96.36%
Epoch 8/10, Train Loss: 0.0963, Validation Loss: 0.1048, Validation Accuracy: 94.55%
Epoch 9/10, Train Loss: 0.0901, Validation Loss: 0.0776, Validation Accuracy: 96.36%
Epoch 10/10, Train Loss: 0.0769, Validation Loss: 0.0793, Validation Accuracy: 96.36%
Model fine-tuning complete and saved!


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/10, Train Loss: 2.3524, Validation Loss: 2.2436, Validation Accuracy: 14.55%
Epoch 2/10, Train Loss: 1.6969, Validation Loss: 1.1433, Validation Accuracy: 65.45%


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/10, Train Loss: 2.1555, Validation Loss: 1.7818, Validation Accuracy: 41.82%
Epoch 2/10, Train Loss: 1.2971, Validation Loss: 0.9896, Validation Accuracy: 67.27%
Epoch 3/10, Train Loss: 0.9296, Validation Loss: 0.5916, Validation Accuracy: 87.27%
Epoch 4/10, Train Loss: 0.7186, Validation Loss: 0.2760, Validation Accuracy: 96.36%
Epoch 5/10, Train Loss: 0.5095, Validation Loss: 0.2283, Validation Accuracy: 94.55%
Epoch 6/10, Train Loss: 0.3705, Validation Loss: 0.5605, Validation Accuracy: 83.64%
Epoch 7/10, Train Loss: 0.3945, Validation Loss: 0.2881, Validation Accuracy: 89.09%
Epoch 8/10, Train Loss: 0.3664, Validation Loss: 0.1907, Validation Accuracy: 94.55%
Epoch 9/10, Train Loss: 0.3923, Validation Loss: 0.1794, Validation Accuracy: 92.73%
Epoch 10/10, Train Loss: 0.2506, Validation Loss: 0.2514, Validation Accuracy: 90.91%
Model fine-tuning complete and saved!


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.0001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/10, Train Loss: 2.3890, Validation Loss: 2.3931, Validation Accuracy: 9.09%
Epoch 2/10, Train Loss: 2.3665, Validation Loss: 2.3544, Validation Accuracy: 23.64%
Epoch 3/10, Train Loss: 2.2326, Validation Loss: 2.1737, Validation Accuracy: 27.27%
Epoch 4/10, Train Loss: 1.9291, Validation Loss: 1.8951, Validation Accuracy: 29.09%
Epoch 5/10, Train Loss: 1.6290, Validation Loss: 1.4489, Validation Accuracy: 52.73%
Epoch 6/10, Train Loss: 1.3880, Validation Loss: 1.1952, Validation Accuracy: 56.36%
Epoch 7/10, Train Loss: 1.1941, Validation Loss: 0.9176, Validation Accuracy: 74.55%
Epoch 8/10, Train Loss: 0.9910, Validation Loss: 0.7975, Validation Accuracy: 80.00%
Epoch 9/10, Train Loss: 0.9156, Validation Loss: 0.7030, Validation Accuracy: 80.00%
Epoch 10/10, Train Loss: 0.8609, Validation Loss: 0.7267, Validation Accuracy: 81.82%
Model fine-tuning complete and saved!


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=50, lr=0.0001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/50, Train Loss: 2.3973, Validation Loss: 2.3919, Validation Accuracy: 9.09%
Epoch 2/50, Train Loss: 2.3732, Validation Loss: 2.3461, Validation Accuracy: 32.73%
Epoch 3/50, Train Loss: 2.2257, Validation Loss: 2.1465, Validation Accuracy: 36.36%
Epoch 4/50, Train Loss: 1.8926, Validation Loss: 1.7667, Validation Accuracy: 41.82%
Epoch 5/50, Train Loss: 1.5750, Validation Loss: 1.4436, Validation Accuracy: 50.91%
Epoch 6/50, Train Loss: 1.3164, Validation Loss: 1.3430, Validation Accuracy: 47.27%
Epoch 7/50, Train Loss: 1.2026, Validation Loss: 1.0567, Validation Accuracy: 60.00%
Epoch 8/50, Train Loss: 1.0817, Validation Loss: 0.9182, Validation Accuracy: 69.09%
Epoch 9/50, Train Loss: 1.0272, Validation Loss: 0.8743, Validation Accuracy: 67.27%
Epoch 10/50, Train Loss: 0.9688, Validation Loss: 0.8679, Validation Accuracy: 65.45%
Epoch 11/50, Train Loss: 0.8389, Validation Loss: 0.7738, Validation Accuracy: 72.73%
Epoch 12/50, Train Los

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=10, lr=0.001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/10, Train Loss: 2.3889, Validation Loss: 2.3642, Validation Accuracy: 18.18%
Epoch 2/10, Train Loss: 2.2462, Validation Loss: 2.0985, Validation Accuracy: 18.18%
Epoch 3/10, Train Loss: 1.7534, Validation Loss: 1.5577, Validation Accuracy: 69.09%
Epoch 4/10, Train Loss: 1.1531, Validation Loss: 0.7308, Validation Accuracy: 87.27%
Epoch 5/10, Train Loss: 0.5179, Validation Loss: 0.2836, Validation Accuracy: 94.55%
Epoch 6/10, Train Loss: 0.2893, Validation Loss: 0.1357, Validation Accuracy: 98.18%
Epoch 7/10, Train Loss: 0.1607, Validation Loss: 0.1070, Validation Accuracy: 94.55%
Epoch 8/10, Train Loss: 0.1144, Validation Loss: 0.0685, Validation Accuracy: 98.18%
Epoch 9/10, Train Loss: 0.1119, Validation Loss: 0.1107, Validation Accuracy: 96.36%
Epoch 10/10, Train Loss: 0.0831, Validation Loss: 0.0846, Validation Accuracy: 96.36%
Model fine-tuning complete and saved!


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=50, lr=0.0001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/50, Train Loss: 2.3992, Validation Loss: 2.3973, Validation Accuracy: 9.09%
Epoch 2/50, Train Loss: 2.3961, Validation Loss: 2.3953, Validation Accuracy: 9.09%
Epoch 3/50, Train Loss: 2.3922, Validation Loss: 2.3922, Validation Accuracy: 9.09%
Epoch 4/50, Train Loss: 2.3868, Validation Loss: 2.3873, Validation Accuracy: 16.36%
Epoch 5/50, Train Loss: 2.3783, Validation Loss: 2.3798, Validation Accuracy: 18.18%
Epoch 6/50, Train Loss: 2.3666, Validation Loss: 2.3685, Validation Accuracy: 23.64%
Epoch 7/50, Train Loss: 2.3488, Validation Loss: 2.3528, Validation Accuracy: 40.00%
Epoch 8/50, Train Loss: 2.3243, Validation Loss: 2.3296, Validation Accuracy: 45.45%
Epoch 9/50, Train Loss: 2.2869, Validation Loss: 2.2964, Validation Accuracy: 45.45%
Epoch 10/50, Train Loss: 2.2296, Validation Loss: 2.2494, Validation Accuracy: 41.82%
Epoch 11/50, Train Loss: 2.1527, Validation Loss: 2.1879, Validation Accuracy: 43.64%
Epoch 12/50, Train Loss:

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


num_classes = len(os.listdir('/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/train'))

# Initialize the model
model = FaceRecognitionModel(num_classes=num_classes).to(device)

sample_input_shape = (1, 3, 128, 128)  # Batch size 1, RGB image of size 128x128
model.initialize_fc1(sample_input_shape)

print("Model fc1 layer initialized successfully!")


model.fine_tune(train_loader, val_loader, num_epochs=50, lr=0.00001)

# Save the fine-tuned model
torch.save(model.state_dict(), 'face_recognition_model.pth')
print("Model fine-tuning complete and saved!")





Model fc1 layer initialized successfully!
Epoch 1/50, Train Loss: 2.3968, Validation Loss: 2.3982, Validation Accuracy: 9.09%
Epoch 2/50, Train Loss: 2.3966, Validation Loss: 2.3981, Validation Accuracy: 9.09%
Epoch 3/50, Train Loss: 2.3964, Validation Loss: 2.3980, Validation Accuracy: 9.09%
Epoch 4/50, Train Loss: 2.3961, Validation Loss: 2.3979, Validation Accuracy: 9.09%
Epoch 5/50, Train Loss: 2.3959, Validation Loss: 2.3978, Validation Accuracy: 9.09%
Epoch 6/50, Train Loss: 2.3957, Validation Loss: 2.3977, Validation Accuracy: 9.09%
Epoch 7/50, Train Loss: 2.3955, Validation Loss: 2.3975, Validation Accuracy: 9.09%
Epoch 8/50, Train Loss: 2.3952, Validation Loss: 2.3974, Validation Accuracy: 9.09%
Epoch 9/50, Train Loss: 2.3950, Validation Loss: 2.3973, Validation Accuracy: 9.09%
Epoch 10/50, Train Loss: 2.3947, Validation Loss: 2.3971, Validation Accuracy: 9.09%
Epoch 11/50, Train Loss: 2.3945, Validation Loss: 2.3970, Validation Accuracy: 9.09%
Epoch 12/50, Train Loss: 2.3942,

In [ ]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

# Mount Google Drive (if using it)
from google.colab import drive
drive.mount('/content/drive')

# Define the image folder and output folder (adjust paths)
image_folder = "/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/data/val/Animesh"  # Update with your folder path
output_folder = "/content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face"  # Update with your folder path
os.makedirs(output_folder, exist_ok=True)

# Define class labels
class_labels = ["Animesh", "Ankita", "Anmol", "Deependra", "Narayan", "Pankaj", "Sahil", "Shivam", "Sonal", "Suraj", "Vidya"]
# Load the fine-tuned model
model = FaceRecognitionModel(num_classes=len(class_labels))  # Update number of classes
# # Dynamically initialize fc1 with the same input shape used during training
sample_input_shape = (1, 3, 128, 128)  # Replace with your input dimensions
model.initialize_fc1(sample_input_shape)
model.load_state_dict(torch.load("/content/face_recognition_model.pth", map_location="cpu"))  # Adjust path
model.eval()

# Image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to match model input
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

# Function to annotate an image
def annotate_image(image, label):
    annotated_image = image.copy()
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(annotated_image, label, (10, 30), font, 1, (0, 255, 0), 2, cv2.LINE_AA)
    return annotated_image

# Process each image in the folder
for filename in os.listdir(image_folder):
    file_path = os.path.join(image_folder, filename)

    # Skip non-image files
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    # Load and preprocess the image
    image = Image.open(file_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)

    # Perform inference
    with torch.no_grad():
        outputs = model(input_tensor)
        _, predicted = torch.max(outputs, 1)
        predicted_label = class_labels[predicted.item()]

    # Convert to OpenCV format for annotation
    cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    annotated_image = annotate_image(cv_image, predicted_label)

    # Save the annotated image to the output folder
    output_path = os.path.join(output_folder, f"annotated_{filename}")
    cv2.imwrite(output_path, annotated_image)

    print(f"Processed and saved: {output_path}")

print("All images processed and annotated!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed and saved: /content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face/annotated_1676639882315.png
Processed and saved: /content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face/annotated_1720390672972.jpg
Processed and saved: /content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face/annotated_IMG_20230217_123800.jpg
Processed and saved: /content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face/annotated_IMG_20221024_091759.jpg
Processed and saved: /content/drive/Shareddrives/CNN_Project/Attendance_Management/Photos2026/Animesh_Face/annotated_1723174231715.jpg
All images processed and annotated!
